In [29]:
#imports
import random

In [30]:

class timetable:
    def __init__(self,classes,Norooms,NoCourses,Professors, populationSize ) -> None:
        self.classes = classes
        self.Norooms = 16
        self.NoCourses = NoCourses
        self.NoSections = 0
        self.SectionTypes = 2
        self.Professors = Professors
        self.Days = 5
        self.TimeSlots = 6
        self.rooms = 16
        self.population = []
        self.populationSize = populationSize

        #self.roomsTypes = random.sample(range(1, self.Norooms + 1), self.Norooms)
        self.roomTypes = [random.choice([0, 1]) for _ in range(self.rooms)]
        print(self.roomTypes)

    def GetRandom(self,upperBound):
        numbers = list(range(1, upperBound + 1))
        random.shuffle(numbers)
        return numbers

    def generatePopulation(self,  Course,Sec,Type):
        population = []
        for i in range(self.populationSize):
            population.append(self.generateOneChromosome(Course,Sec,Type))
       
        return population
            
    def getHighest(self,arr1, arr2):
        max = 0
        pos =None
        #print("lens ", len(arr1), len(arr2))
        for i,j in enumerate(arr2):
            
            if j > max:
                max = j
                pos = i

        #print("max",max,pos)
        return arr1[pos], max


    def generateClass(self):
        #get classes list
        CourseList = self.GetRandom(self.NoCourses)
        #print("courfse list ", CourseList)
        SectionList = [random.choice([0, 7]) for _ in range(self.NoCourses)] #keeping max sections as 7
        population = []
        timetable_ = []
        prob = 5
        for i,j in enumerate(CourseList):
            for k in range(SectionList[i]):
                #population = []
                #print("our tt: ", timetable_)
                population = self.generatePopulation(j,k,0)
                p, timetablepop = time.GeneticAlgo(population,timetable_)
                #print("TTP",timetablepop)
                score = [time.FitnessFunc(timetablepop,c) for c in timetablepop]
                pos, m = self.getHighest(timetablepop, score)
                if m == 1.0:
                    timetable_.append(pos)
                
                #add lab to course
                num = random.random() 
                
                if num <= 0.05:
                    
                    #population = []
                    population = self.generatePopulation(j, k,1)
                    
                    p, timetablepop = time.GeneticAlgo(population,timetable_)
                    score = [time.FitnessFunc(timetablepop,c) for c in timetablepop]
                    pos, m = self.getHighest(timetablepop, score)
                    if m == 1.0:
                        timetable_.append(pos)
                    #timetable_.append(timetablepop)
                    
        return timetable_
    

    
    def generateOneChromosome(self, Course, Sec, Type):
        chromosome = []
        #add course
        chromosome.append( format(Course,'b') )
        #add type
        
        chromosome.append(format(Type,'b') )
        #add Section
        
        chromosome.append(format(Sec,'b') )
        #add Section type
        num = random.random()
        prob = 0.2
        
        if prob < num:
            chromosome.append(format(1,'b'))
        else:
            chromosome.append(format(0,'b') )

        #add Professor
        num = random.randint(1,self.Professors) 
        chromosome.append(format(num,'b') )
        #add Day1
        num = random.randint(1,self.Days) 
        chromosome.append(format(num,'b') )
        #add timeslot1
        num = random.randint(1,self.TimeSlots) 
        chromosome.append(format(num,'b') )
        #add classroom1
        num = random.randint(1,self.rooms) 
        chromosome.append(format(num,'b') )
        #add Day2
        num = random.randint(1,self.Days) 
        chromosome.append(format(num,'b') )
        #add timeslot2
        num = random.randint(1,self.TimeSlots) 
        chromosome.append(format(num,'b') )
        #add classroom2
        num = random.randint(1,self.Norooms) 
        chromosome.append(format(num,'b') )

        return chromosome
    
    def SinglePtCrossover(self,p1,p2,pt):
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
        return c1,c2
    def performCrossover(self,p1, p2, r_cross):
        # children are copies of parents by default
        c1, c2 = p1.copy(), p2.copy()
        # check for recombination
        #if random.random() < r_cross:
        # select crossover point that is not on the end of the string
        pt = random.randint(2, len(p1)-2)
        crossover_points = random.sample(range(2, len(p1)-1), 2)
        pt = sorted(crossover_points)
        # perform crossover
        for i in pt:
            c1,c2 = self.SinglePtCrossover(c1,c2,i)
        #print("parent1: ", p1, "parent 2: ", p2)
        #print("child1: ", c1, "child 2: ", c2)
        return [c1, c2]
    
    def mutation(self,bitstring, r_mut):
        #print("bitstring", bitstring)
        for i, j in enumerate(bitstring):
            if (i > 2):
                #chrck if mutation to be done
                if random.random() < r_mut:
                    #print("mutating:", bitstring)
                    lenBit = len(j)
                    y = random.randint(0,lenBit-1)
                    bit_list = list(bitstring[i])
                    bit = bit_list[y]
                    if bit  == '0' :
                        bit_list[y] =  '1'
                    else:
                        bit_list[y] = '0'
                                    
                    bitstring[i] = "".join(bit_list)
               
        #print("mutated: ", bitstring)



    def selection(self,pop, scores, k=3):
        # first random selection
        selection_ix = random.randint(0, len(pop)-1)
        for ix in random.sample(range(len(pop)), k-1):
            # check if better (e.g. perform a tournament)
            
            ix -= 1
            if scores[ix] > scores[selection_ix]:
                selection_ix = ix
        return pop[selection_ix]
    
    
    def FitnessFunc(self, population, chromosome):
        
        clash = 0
        
        #Classes can only be scheduled in free classrooms.
        #check for day 1
        room1 = chromosome[7]
        room2 = chromosome[10]
        for i in population:
            #print(room1, i[7])
            if i !=chromosome:
                if room1 == i[7] and ( i[6] == chromosome[6] and i[5] == chromosome[5] )   :
                    clash +=1
                    #print("no same")
                if room2 == i[10] and (i[8] == chromosome[8] and i[9] == chromosome[9] ) :
                    clash+=1
                    #print("no same")
        
        #A classroom should be big enough to accommodate the section. There should be two
        #categories of classrooms: classroom (60) and large hall (120).
        room1 = int(chromosome[7],2)
        room2  = int(chromosome[10],2)
        sectionType = int(chromosome[3],2)
        #print(room1, room2)
        roomType1 = self.roomTypes[room1-1]
        roomType2 = self.roomTypes[room2-1]
        if sectionType == 1 and roomType1 != 1:
            clash+=1
            #print("big classroon")
        if sectionType == 1 and roomType2 != 1:
            clash +=1
            #print("big classroom 2")
        
        #A professor should not be assigned two different lectures at the same time.
        prof = chromosome[4]
        for i in population:
            if i != chromosome:
                #check for same time              
                if i[4] == prof and (( i[6] == chromosome[6] and i[5] == chromosome[5] ) or  (i[8] == chromosome[8] and i[9] == chromosome[9] ) ):
                    clash +=1
                    
                    #print("no prof ass")

        #The same section cannot be assigned to two different rooms at the same time. 
        sec = chromosome[2]
        for i in population:
            if i !=chromosome and sec==i[2] and  ( ( i[6] == chromosome[6] and i[5] == chromosome[5] ) or  (i[8] == chromosome[8] and i[9] == chromosome[9] ) ):
                clash += 1
                
                #print("no same sec")
        
        #No professor can teach more than 3 courses.
        courseCount = 0
        course = []
        for  i in population:
            if  i != chromosome and prof == i[4] and i[0] not in course:
                course.append(i[0])
        if len(course) >3:
            clash +=1
            #print("no same 3course")
        #No section can have more than 5 courses in a semester.
        sec = chromosome[2]
        course = []
        for i in population:
            if i !=chromosome and sec==i[2] and i[0] not in course:
                course.append(i[0])
        if (len(course)) >5:
            clash += 1
            #print("no same clas 5 c")

        #Each course would have two lectures per week not on the same or adjacent days.
        if (chromosome[1] == 0):
            chCourse = chromosome[0]
            day1 = int(chromosome[5],2)
            day2 = int(chromosome[8],2)
            if (day1 + 1 == day2 or day1 - 1 == day2 or day1 == day2):
                clash+=1
                #print("not adj days")
            
        #Lab lectures should be conducted in two consecutive slots.

        typ = chromosome[1]
        
        if typ == '1':
            #print("conseccc")
            day1 = int(chromosome[5],2)
            day2 = int(chromosome[8],2)
            time1 = int (chromosome[6], 2)
            time2 = int(chromosome[9], 2)
            if not (time1 +1 == time2):
                clash += 1
                #print("not equal time")
            #print(time1, time2)
            if not (day1 == day2 ):
                clash +=1
                #print("consec")
        #if clash != 0:
            #soft constraints
            #All the theory classes should be taught in the morning session and all the lab sessions
            # should be done in the afternoon session.
            # if chromosome[1] == '0': #if theory
            #     time1 = int (chromosome[6], 2)
            #     time2 = int(chromosome[9], 2)
            #     if not (time1 == 1 or time1 == 2 or time1 == 3):
            #         clash +=1
            # else:
            #     time1 = int (chromosome[6], 2)
            #     time2 = int(chromosome[9], 2)
            #     if not (time1 == 4 or time1 == 5 or time1 == 6):
            #         clash +=1

            # • Teachers/students may be facilitated by minimizing the number of floors they have to
            # traverse. That is, as much as possible, scheduled classes should be on the same floor for
            # either party.
            #here i am assuming that each floor has 4 rooms. Floor 1 has rooms 1 2 3 4 and so on
            # room1 = int (chromosome[7], 2)
            # room2 = int( chromosome[10], 2)

            # floor1 = (room1 - 1) // 4 + 1
            # floor2 = (room2 - 1)// 4+1
            # if floor1 != floor2:
            #     clash +=1

            # • A class should be held in the same classroom across the whole week.
            # room1 = int (chromosome[7], 2)
            # room2 = int( chromosome[10], 2)
            # if room1 != room2:
            #     clash +=1
            # • Teachers may prefer longer blocks of continuous teaching time to minimize
            # interruptions and maximize productivity except when the courses are different.
            
        fitness = 1/(float(clash) +1)
        return fitness
  
    def GeneticAlgo(self, pop,timeTabl, gens = 6000):
            #print(gens)
            #print(pop)
            timetablePop = []
            # keep track of best solution
            best, best_eval = 0, self.FitnessFunc (timeTabl, pop[0])
            n_pop = len(pop)
            r_cross = 0.7
            r_mut = 0.02
            # enumerate generations
            Changecounter = 0
            for gen in range(gens):
                if Changecounter < 60:
                    r_mut += 0.01
                # evaluate all candidates in the population
                scores = [self.FitnessFunc(timeTabl,c) for c in pop] 
                # check for new best solution
                for i in range(len(pop)):
                    if scores[i] >= best_eval:
                        Changecounter = 0
                        best, best_eval = pop[i], scores[i]
                        #print(">%d, new best f(%s) = %.3f" % (gen, pop[i], scores[i]))
                        timetablePop.append(pop[i])
                        if scores[i] == 1.0: #stopping condition
                            #print("stopping")
                            return pop, timetablePop
                # select parents
                selected = [self.selection(pop, scores) for _ in range(n_pop)]
                # create the next generation
                children = list() 
                for i in range(0, n_pop, 2):
                    # get selected parents in pairs
                    p1, p2 = selected[i], selected[i+1]
                    # crossover and mutation
                    for c in self.performCrossover(p1, p2, r_cross):
                        # mutation
                        self.mutation(c, r_mut)
                        # store for next generation
                        children.append(c)
                # replace population
                pop = children
                Changecounter +=1
            print("not stopping")
            return pop, timetablePop
    def printTimeTable(self,pop):
        timeslots = ["08:30-09:50",	"10:00-11:20",	"11:30-12:50",	"01:00-02:00"	,"02:00-03:20", "03:30-04:50"]
        
        for i in  range(1,6):
            print(f"Day {i}")
            print(timeslots)
            
            for j in range(1,7):
                print(timeslots[j-1]," : ")
                for z,p in enumerate(pop):
                    found = False
                    #if scores[z] == 1.0:             
                        
                    if int(p[5],2)== i and int(p[6],2) == j: #first class
                        #print(j)
                        found = True
                        Cn = int(p[0],2)
                        cnT = int(p[1],2)
                        Sec = int(p[2],2)
                        room = int(p[7],2)
                        print(f"C{Cn}-{Sec}-{cnT}-{room}" )
                        print("++++++++++++++++")
                        #break  # Stop searching for this period once found
                    elif int(p[8], 2)== i and int(p[9],2) == j:
                        found = True
                        Cn = int(p[0],2)
                        cnT = int(p[1],2)
                        Sec = int(p[2],2)
                        room = int(p[10],2)
                        print(f"C{Cn}-{Sec}-{cnT}-{room}")
                        print("++++++++++++++++")
                        #break  # Stop searching for this period once found

                    
                    #print("     ", end="")  # If no class is found for the timeslot

            #print("|")

        #print("")


time = timetable(0,16,8,20,5000)
#pop = time.generatePopulation(100)
timetable_ = []
#score = [time.FitnessFunc(timetable_,c) for c in pop]
TT= time.generateClass()
print(TT)
print("CHECKING+++++++++++++++++++++++++++")
score = [time.FitnessFunc(TT,c) for c in TT]
print(score)
time.printTimeTable(TT)
#print(score)

#p, timetablepop = time.GeneticAlgo(pop,timetable_,10)
#score = [time.FitnessFunc(timetablepop,c) for c in timetablepop]
#print(sorted(score, reverse=True))
#print(time.printTimeTable(timetablepop, score))

[0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0]
[['10', '0', '0', '0', '10001', '100', '11', '111', '100', '10', '1000'], ['10', '0', '1', '1', '10001', '1', '11', '1001', '100', '1', '1000'], ['10', '0', '10', '1', '10000', '10', '1', '1100', '10', '110', '1111'], ['10', '0', '11', '0', '11', '100', '101', '1111', '100', '100', '1110'], ['10', '0', '100', '1', '1011', '100', '10', '1001', '101', '11', '100'], ['10', '0', '101', '1', '1', '1', '100', '1001', '10', '1', '1100'], ['10', '0', '110', '0', '110', '1', '110', '1011', '11', '100', '11'], ['110', '0', '0', '0', '100', '100', '100', '1011', '101', '11', '1'], ['110', '0', '1', '0', '1001', '101', '101', '1010', '101', '110', '1000'], ['110', '0', '10', '1', '110', '1', '100', '1110', '100', '1', '1000'], ['110', '1', '10', '1', '10001', '11', '101', '1100', '11', '110', '1111'], ['110', '0', '11', '0', '101', '101', '110', '10', '1', '101', '1'], ['110', '0', '100', '0', '1011', '101', '100', '1100', '10', '11', '1100'], ['110

In [31]:
print(1/4)

0.25
